In [9]:
import numpy as np 
import bagpipes as pipes

from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load CANDELS GOODS South photometry from the Guo et al. (2013) catalogue. """

    # load up the relevant columns from the catalogue.
    cat = np.loadtxt("hlsp_candels_hst_wfc3_goodss-tot-multiband_f160w_v1-1photom_cat.txt",
                     usecols=(10, 13, 16, 19, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55,
                              11, 14, 17, 20, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56))
    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    # Extract the object we want from the catalogue.
    fluxes = cat[row, :15]
    fluxerrs = cat[row, 15:]

    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list.txt", dtype="str")

#dictionary for fit instructions
exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp   
fit_instructions["dust"] = dust

phot = load_goodss(1)  # or 2 or 3
print(type(phot))
print(phot.shape)
print(phot)



<class 'numpy.ndarray'>
(15, 2)
[[8.805800e-01 4.402900e-02]
 [1.572470e+00 7.862350e-02]
 [6.561540e+00 3.280770e-01]
 [1.567080e+01 7.835400e-01]
 [2.316310e+01 1.158155e+00]
 [0.000000e+00 9.900000e+99]
 [0.000000e+00 9.900000e+99]
 [3.790730e+01 1.895365e+00]
 [5.483620e+01 2.741810e+00]
 [0.000000e+00 9.900000e+99]
 [0.000000e+00 9.900000e+99]
 [7.636190e+01 7.636190e+00]
 [5.892980e+01 5.892980e+00]
 [4.955860e+01 4.955860e+00]
 [3.596970e+01 3.596970e+00]]


In [10]:
IDs = np.arange(1, 4, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="guo_cat")

fit_cat.fit(verbose=False)

# BAGPIPES fitting for my mini catalgoue

In [1]:
#trying this but for my mini cat of n galaxies

import numpy as np 
import bagpipes as pipes
from astropy.table import Table
from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load catalogue. """

    # load up the relevant columns from the catalogue.
    cat = Table.read("mini_cat_20galaxiesonly.fits") 

    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    filters = Table.read('wavelengths_filters.csv', format='csv')
    # Extract filter names
    filters_list = filters['flux_filter'].tolist()

    # Build column names for fluxes and flux errors
    flux_columns = [f'FLUX_APER_{f}_aper_corr_Jy' for f in filters_list]
    fluxerr_columns = [f'FLUXERR_APER_{f}_loc_depth_10pc_Jy' for f in filters_list]
     # Extract fluxes and errors for this galaxy from the catalogue
    fluxes = np.array([cat[row][col] for col in flux_columns])
    fluxes = np.where(fluxes > 0, fluxes, np.nan)
    fluxerrs = np.array([cat[row][col] if not np.ma.is_masked(cat[row][col]) else np.nan for col in fluxerr_columns])


    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0) or np.isnan(photometry[i, 0]) or np.isnan(photometry[i, 1]):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list1.txt", dtype="str")

#dictionary for fit instructions
# exp = {}                                  
# exp["age"] = (0.1, 15.)
# exp["tau"] = (0.3, 10.)
# exp["massformed"] = (1., 15.)
# exp["metallicity"] = (0., 2.5)

lognorm = {}                       # lognormal SFH
lognorm["tmax"] = (0.01, 15) # these will default to a flat prior probably
lognorm['fwhm'] = (0.01, 15) 
lognorm['metallicity'] = (0., 2.5)  # metallicity: Z_Solar
lognorm["massformed"] = (5., 12.)  # Log_10 total stellar mass formed: M_Solar

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["lognormal"] = lognorm
fit_instructions["dust"] = dust


In [2]:
IDs = np.arange(1, 21, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="mini_cat_20_gals_lognormal")

fit_cat.fit(verbose=False)


Bagpipes: fitting object 1

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************
 ln(ev)=  -94.630967686922986      +/-   8.0156097709404669E-002
 Total Likelihood Evaluations:         2545
 Sampling finished. Exiting MultiNest
Rank 0 for 1 True

Completed in 17.9 seconds.

Generating 400 samples from posterior. Total samples: 400


/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: invalid value encountered in multiply
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: overflow encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/nvme/scratch/software/anaconda3/envs/tharvey_galfind/lib/python3.10/site-packages/numpy/lib/function_base.py:1448: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/chemical_enrichment_history.py:121: RuntimeWarning: invalid value encountered in mul

'nebular'
Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.376      1.098      1.718
lognormal:fwhm                 0.223      0.826      1.718
lognormal:massformed           6.498      9.230     11.131
lognormal:metallicity          0.451      1.286      2.134
lognormal:tmax                 7.476     11.314     13.866
redshift                       3.325      6.611      8.872




Bagpipes: 1 out of 20 objects completed.

Bagpipes: fitting object 2

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************
 ln(ev)=   118.43807641301876      +/-   8.3479039285324172E-002
Rank 0 for 2 True

Completed in 22.3 seconds.

 Total Likelihood Evaluations:         2875
 Sampling finished. Exiting MultiNest
Generating 400 samples from posterior. Total samples: 400


/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: invalid value encountered in multiply
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:272: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  tau, t0 = fsolve(lognorm_equations, (tau_guess, t0_guess),
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: overflow encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/nvme/scratch/software/anaconda3/envs/tharvey_galfind/lib/python3.10/si

...

/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: invalid value encountered in multiply
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:272: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  tau, t0 = fsolve(lognorm_equations, (tau_guess, t0_guess),
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: overflow encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/nvme/scratch/software/anaconda3/envs/tharvey_galfind/lib/python3.10/si

'nebular'
Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.434      1.159      1.720
lognormal:fwhm                 0.221      0.777      1.710
lognormal:massformed           6.375      8.671     10.710
lognormal:metallicity          0.462      1.261      2.080
lognormal:tmax                 6.648     10.987     13.531
redshift                       3.742      6.816      9.156


Bagpipes: 19 out of 20 objects completed.

Bagpipes: fitting object 20

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    6
 *****************************************************
Rank 0 for 20 ln(ev)=   117.30330277110987      +/-   8.1432487374515639E-002
 Total Like

/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: invalid value encountered in multiply
  self.component_sfrs[name] *= desired_mass/mass_norm
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:272: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  tau, t0 = fsolve(lognorm_equations, (tau_guess, t0_guess),
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/star_formation_history.py:112: RuntimeWarning: overflow encountered in scalar divide
  self.component_sfrs[name] *= desired_mass/mass_norm
/nvme/scratch/software/anaconda3/envs/tharvey_galfind/lib/python3.10/si

'nebular'
Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.327      1.045      1.639
lognormal:fwhm                 0.183      0.682      1.638
lognormal:massformed           5.965      8.392     10.805
lognormal:metallicity          0.442      1.238      2.030
lognormal:tmax                 6.462     10.881     13.953
redshift                       3.387      6.498      8.900


Bagpipes: 20 out of 20 objects completed.


/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py:674: RuntimeWarning: divide by zero encountered in log10
  self.uvj = -2.5*np.log10(self._calculate_photometry(0., uvj=True))
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py:691: OptimizeWarning: Covariance of the parameters could not be estimated
  self.beta_C94 = np.array([curve_fit(beta_slope_power_law_func, wav_obs_C94, f_lambda_obs_C94, maxfev = 10_000)[0][1]])
/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/models/model_galaxy.py:697: RuntimeWarning: divide by zero encountered in log10
  self.m_UV = np.array([-2.5 * np.log10((f_lambda_1500 * ((1_500. * (1 + model_comp["redshift"]) * u.AA) ** 2) / const.c).to(u.Jy).value) + 8.9]) # observed frame
Traceback (most recent call last):
  File "/raid/scratch/work/tharvey/github_bagpipes_repo/bagpipes/bagpipes/fitting/fit.py", line 396, in fit
    self.posterior.get_advanced_quantities()
  Fi